<a href="https://colab.research.google.com/github/maltehueckstaedt/gerit_matching/blob/main/gerit_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GERIT PRE-Matching mit OpenAI-Embeddings

Im folgenden wird ein Notebook entworfen, dass das derzeitige Matching der HEX der Daten mit GERIT-Organisationen erleichtern soll. Im Derzeitigen Stand geht folgendermaßen vor:

Es wird von einem klassischen Cleaning der Organisations-Variable, wie sie aus den Scrapiung kommt, abgesehen, weil es im vergleich zum folgenden Vorgehen (und angesichts des Umstandes, dass wir ja die *wahren* Organisationennamen schon kennen) uneffizient und die Gefahr von Artfacten höher erscheint.

Statt eines klassischen Cleanings mit Hilfe von LLMs wird eine Funktion implementiert,die nach Entsprechungen in der Liste der GERIT-Organisationen. Dafür werden mit OpenAI Embeddings Vektor-Repräsentationen aller GERIT-Namen erstellt. Ein *„Retriever“* durchsucht diese Vektoren, um die besten Übereinstimmungen zu einem gegebenen Namen (also den Organisationsnamen aus dem Scraping) zu finden. Das Modell kann z. B. erkennen, dass „*Institut für Biologie*“ und „*Department Biologie*“ wahrscheinlich dasselbe meinen.

Für jede Organisation wird überprüft, ob es eine exakte Übereinstimmung gibt. Falls ja, wird der offizielle GERIT-Name übernommen. Falls keine exakte Übereinstimmung gefunden wird, kommt ein Fuzzy Matching zum Einsatz, bei dem ein kosinusbasierter Score berechnet wird (0 = völlig unterschiedliche Begriffe, 1 = perfekte Übereinstimmung). Ist der Score hoch genug (aktuell: > 0.5), wird der bestpassende GERIT-Treffer übernommen. Falls keine passende Entsprechung gefunden wird, bleibt der Wert NA.

Vorteile: Die Funktion spart Zeit – es werden derzeit 20.000 Zeilen und 250 unique Werte in etwa 1,5 Minuten umcodiert. Zudem ist kein manuelles Coding oder eine zusätzliche Aufbereitung der Organisationsvariablen nötig. Mit dem Score-Cutoff kann weiterhin die Konservativität des tools gesteuert werden.

Nachteile: Die Qualität der Zuordnung muss geprüft werden, und es bleibt die Frage, wie sich das neue >>Cleaning>> oder Vor-Matching auf nachfolgende Analysen auswirkt.

## Programmierung der Funktion

### Einrichtung Arbeitspfad

In einem ersten Schritt überprüfen wir unseren derzeitgen Arbeitspfad (in dem Fall das wir lokal arbeiten).

In [1]:
import os
os.getcwd()
os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/gerit_matching")

### Laden und checke der Daten

Wir laden einerseits testweise der Daten (inkl. der Organisationsvariable) der HHU, andererseits die GERIT-Daten der hhu.

In [2]:
import pandas as pd

db_hhu = pd.read_csv("data\hhu_db_raw.csv")
#db_hhu = db_hhu.sample(n=200, random_state=42)
hhu_gerit = pd.read_excel("data/hhu_gerit.xlsx")

Wir zeigen in folgenden die Spaltennamen, den Typ der Variable `organisation_mehrere` und ihre uniquen Werte an.

In [3]:
db_hhu.columns
print(db_hhu["organisation_mehrere"].dtype)
print(db_hhu["organisation_mehrere"].unique())

object
[nan 'Institut für Modernes Japan' 'Juristische Fakultät'
 'Juristische Fakultät ; Fakultäten'
 'Wirtschaftswissenschaftliche Fakultät'
 'Entwicklungs- und Molekularbiologie der Pflanzen ; Genetik ; Funktionelle Genomforschung der Mikroorganismen'
 'Pflanzliche Zellbiologie und Biotechnologie ; Funktionelle Genomforschung der Mikroorganismen'
 'Department Biologie ; Botanik' 'Botanik'
 'Department Biologie ; Fakultäten ; Botanik'
 'Wissenschaftliche Einrichtung Mathematisches Institut ; Department Biologie'
 'Quantitative und Theoretische Biologie CEPLAS ; Wissenschaftliche Einrichtung Mathematisches Institut ; Department Biologie'
 'Department Biologie ; Biochemie der Pflanzen'
 'Neurobiologie ; Stoffwechselphysiologie'
 'Department Biologie ; Neurobiologie ; Stoffwechselphysiologie'
 'Physikalische Biologie'
 'Physikalische Biologie ; Biomolekulare Festkörper-NMR-Spektroskopie'
 'Molekulare Enzymtechnologie ; Funktionelle Genomforschung der Mikroorganismen ; Mikrobiologie'
 'M

Unabhängig davon, welcher Typ die Variable `organisation_mehrere` tatsächlich ist, definieren wandeln wie die Varibale in einen String.

Anschließend erzeugen wir mit `set()` eine leere Menge `unique_values`. `set()` erzeugt eine ungeordnete Sammlung von einzigartigen Elementen. Sets entfernen automatisch doppelte Werte und bieten effiziente Methoden zum Hinzufügen, Entfernen und Überprüfen von Elementen.


Wir nehmen dann den Datensatz `db_hhu` und entfernen alle `NaN`. Anschließend wird für jeden verbleibenden Wert die Funktion `apply()` aufgerufen. Innerhalb der Funktion wird der Text jedes Wertes an den Semikolons geteilt, sodass eine Liste von Werten entsteht. Diese Werte werden dann bereinigt, indem führende und nachstehende Leerzeichen entfernt werden und leere oder mit "NA" (unabhängig von der Groß-/Kleinschreibung) markierte Werte ausgeschlossen werden. Alle bereinigten, nicht leeren und nicht "NA"-Werte werden anschließend dem `unique_values`-Set hinzugefügt, wobei nur einzigartige Werte beibehalten werden.

Abschließend wird zur Information die Anzahl uniquer Organisationen sowie die Merkmale ausgegeben.

In [4]:
import numpy as np

db_hhu["organisation_mehrere"] = db_hhu["organisation_mehrere"].astype(str)

unique_values = set()
db_hhu["organisation_mehrere"].dropna().apply(lambda x: unique_values.update(
    [val.strip() for val in x.split(";") if val.strip() and val.strip().upper() != "NA"]
))

# Ergebnis ausgeben
print(f"Anzahl einzigartiger Werte: {len(unique_values)}")
print(unique_values)

Anzahl einzigartiger Werte: 248
{'Quantitative und Theoretische Biologie CEPLAS', 'Biochemie der Pflanzen', 'Medizinische Fakultät', 'Philosophische Fakultät allgemein', 'Entwicklungs- und Molekularbiologie der Tiere', 'Institut für Neuropathologie', 'Anglistik V - Anglophone Literaturen/Literaturübersetzen', 'Abteilung V Geschichte und Kulturen Osteuropas', 'Institut für Germanistik', 'Abteilung für Lasermassenspektrometrie', 'Mathematische Modellierung biologischer Systeme', 'Studierendenakademie', 'Zentrum für Pharmakologie und Toxikologie', 'C. & O. Vogt-Institut für Hirnforschung', 'Abteilung für Politikwissenschaft', 'Institut für Genetik', 'Lehrstuhl Theoretische Physik II', 'Professur für Volkswirtschaftslehre, insbes. International Economics', 'Funktionelle Genomforschung der Mikroorganismen', 'Frauenklinik', 'Institut für Kunstgeschichte', 'Klinik und Poliklinik für Psychiatrie und Psychotherapie der Heinrich-Heine-Universität Düsseldorf - LVR-Klinikum Düsseldorf', 'D 1.1 Stu

### Lade openAI API

Für das Matching der Organisationsnamen, die aus dem Scraping kommen mit denen der GERIT-Organisationen benötigen wir Embeddings. Dies Embeddings werden mit Hilfe der openAI-API erzeugt. Aus diesem Grund wird der openAI-API-Key im folgenden aus dem Enviorment geladen.


In [5]:
from dotenv import load_dotenv
load_dotenv()

True

Im folgenden wird die OpenAI-API verwendet, um Embeddings der GERIT-Organisationen zu erzeugen und mit hilfe von Langchain eine In-Memory-Vektordatenbank für GERIT-Organisationen zu erstellen.

In einem ersten Schritt wird das OpenAI-Embedding-Modell `text-embedding-3-large` initialisiert. Danach wird eine InMemoryVectorStore-Datenbank erstellt, die diese Vektoren speichert.

Nun werden die GERIT-Organisationen aus dem Pandas-DataFrame (`hhu_gerit["Einrichtung"]`) in `Document`-Objekte umgewandelt. Diese Dokumente enthalten die Namen der Einrichtungen als `page_content`. Schließlich werden diese Dokumente mit ihren Embeddings in die Vektordatenbank eingefügt. Dadurch können später Ähnlichkeitsabfragen durchgeführt werden, um Organisationen mit ähnlichen Namen oder thematischen Schwerpunkten zu finden.

In [6]:
import openai
import pandas as pd
from typing import List
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_core.runnables import chain
from langchain.agents.agent_toolkits import create_retriever_tool

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)

docs = [Document(page_content=einrichtung) for einrichtung in hhu_gerit["Einrichtung"].tolist()]
vector_store.add_documents(docs)

['df973b7b-181c-4f1c-be7b-fbddf0f75661',
 'c61810fa-8d3d-4d91-96b1-9e032969820f',
 '0bebe597-6028-4d57-b717-befe0a04cf68',
 '4bc7834c-c42e-4a07-923b-a3ee02df5d59',
 '03ee48f8-6cab-4089-b436-35a124053d3a',
 '79a145c8-e7f5-4136-8d2b-1be684758c34',
 'bbda7640-2b1c-42af-b446-8d69f2b7c2b3',
 'cb563441-8e72-48e9-b43c-7deaf83a42ed',
 '26efe3a0-ee09-42a9-b514-2d52989a0f76',
 '471884d0-26f6-42c1-b02b-8f2cc0014729',
 'f444ebcf-2f5a-4051-bc5d-c16ca6d3208c',
 '7b839661-0f6c-4b7f-b1cb-5f5861cffc6a',
 'e9c7350d-9eca-4fc3-888a-63c0831d2fad',
 'f4a0b567-5a9a-4f9e-b0fc-48390ff2171e',
 '85000895-b7d3-4b61-99a7-c88cbe9edeba',
 '3c279b3b-a047-45c5-9dda-dfe83d3a5229',
 '68f8b840-db91-4fb6-9d74-aeb458e245da',
 '87d2fd36-3185-421f-9d68-952decb5863a',
 'b42c6e63-1242-414e-b99e-ba6d0f087c7a',
 '8fcb4e10-75b9-4b77-b6f4-09a0595b524c',
 '77b3709b-a895-4c11-99b0-e64d232277cc',
 '0df2259a-52a8-4272-b769-05d4c51759c2',
 'fe06b7a6-048c-43cf-8e25-5218a704c10e',
 '975cfb06-47e8-45c1-a841-b6a3c833ec03',
 '4bca02e5-2987-

In [7]:

# Definiere den Retriever als Chain mit Scores
@chain
def retriever(query: str) -> List[Document]:
    results = vector_store.similarity_search_with_score(query, k=5)
    docs, scores = zip(*results) if results else ([], [])

    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score  # Füge den Score als Metadaten hinzu

    return list(docs)

# Beschreibung für den Retriever
description = (
    "Sucht nach gültigen Eigennamen basierend auf einer ungefähren Eingabe und gibt die Ähnlichkeitsscores zurück. "
    "Falls keine passende Entsprechung gefunden wird, wird 'NA' zurückgegeben."
)

# Retriever-Tool erstellen
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns_with_score",
    description=description,
)


In [8]:
query = "Chemie"
results = retriever.invoke(query)

# Ergebnisse anzeigen
for doc in results:
    print(f"Text: {doc.page_content}, Score: {doc.metadata['score']}")


Text: Fach Chemie, Score: 0.7509366400708087
Text: Fach Physik, Score: 0.5847266610751091
Text: Institut für Physikalische Chemie, Score: 0.5569949476123793
Text: Fach Pharmazie, Score: 0.5564790532357864
Text: Lehrstuhl für Organische Chemie, Score: 0.5424326161142942


In [ ]:
import pandas as pd

def match_unique_organisations(org_series, mapping_dict, retriever):
    """
    Funktion, die nur einzigartige Werte der Organisationen verarbeitet und eine detaillierte Matching-Tabelle erstellt.

    :param org_series: Pandas Series mit den ursprünglichen Organisationen
    :param mapping_dict: Dictionary mit bekannten exakten Zuordnungen
    :param retriever: Retriever zur unscharfen Suche
    :return:
        - recode_dict: Dictionary mit den rekodierten Werten (Originalwert → gematchter Wert)
        - matching_df: DataFrame mit Matching-Details (ursprünglicher Wert, gematchter Wert, Matching-Art, Score)
    """
    print(f"Anzahl einzigartiger Organisationen in den Eingabedaten: {len(org_series)}")

    recode_dict = {}
    matching_data = []  # Liste für die Matching-Ergebnisse

    for org in org_series:
        orgs = [o.strip() for o in org.split(';')]  # Aufteilen bei ";", Leerzeichen entfernen
        matched_orgs = []
        match_details = []  # Speichert Match-Typ und Score für jede Organisation in einer Zeile

        for o in orgs:
            match_type = "Keine Übereinstimmung"
            score = None
            matched_value = None

            # 1. Prüfen, ob eine exakte Übereinstimmung existiert
            if o in mapping_dict:
                matched_value = mapping_dict[o]
                match_type = "Exakt"
                score = 1.0  # Exakte Matches haben Score 1.0
                print(f"Exakte Übereinstimmung gefunden: '{o}' → '{matched_value}'")
            else:
                # 2. Falls keine exakte Übereinstimmung, unscharfe Suche über den Retriever
                results = retriever.invoke(o)  # invoke nutzen, da retriever eine Chain ist

                if results and results[0].metadata.get("score", 0) >= 0.5:
                    matched_value = results[0].page_content
                    score = results[0].metadata["score"]
                    match_type = "Fuzzy"
                    print(f"Fuzzy Match gefunden: '{o}' → '{matched_value}' (Score: {score:.2f})")
                else:
                    print(f"Kein passender Match für '{o}' gefunden.")

            # Speichern des Matches
            matched_orgs.append(matched_value if matched_value else o)  # Falls kein Match, Original behalten
            match_details.append({
                "Ursprünglicher Wert": o,
                "Gematchter GERIT-Wert": matched_value,
                "Matching-Art": match_type,
                "Score": score
            })

        # Rekodierungs-Dictionary speichern
        recode_dict[org] = "; ".join([m for m in matched_orgs if m]) if matched_orgs else None

        # Matching-Details zur Liste hinzufügen
        matching_data.extend(match_details)

    # DataFrame aus den Matching-Daten erstellen
    matching_df = pd.DataFrame(matching_data)

    return recode_dict, matching_df


In [10]:
recode_dict, matching_df = match_unique_organisations(unique_values, mapping_dict, retriever)


NameError: name 'mapping_dict' is not defined

In [11]:
def apply_matched_organisations(org_series, recode_dict):
    """
    Wendet das zuvor berechnete Matching Dictionary auf die originale Spalte an,
    indem alle Organisationen innerhalb eines Eintrags korrekt ersetzt werden.

    :param org_series: Pandas Series mit den ursprünglichen Organisationen
    :param recode_dict: Dictionary mit den rekodierten Werten (Originalwert → gematchter Wert)
    :return: Pandas Series mit den rekodierten Organisationen
    """
    def map_multiple_orgs(org):
        if pd.isna(org):
            return None  # Fehlende Werte beibehalten

        org_list = [o.strip() for o in org.split(';')]  # Mehrere Organisationen aufteilen
        matched_list = [recode_dict.get(o, o) for o in org_list]  # Falls kein Match gefunden wird, Originalwert behalten
        matched_list = [m for m in matched_list if m]  # Leere Werte entfernen

        return "; ".join(matched_list) if matched_list else None  # Falls keine Matches, None zurückgeben

    return org_series.apply(map_multiple_orgs)


In [12]:
db_hhu["gerit_match"] = apply_matched_organisations(db_hhu["organisation_mehrere"], recode_dict)


NameError: name 'recode_dict' is not defined

In [ ]:
matching_df.to_excel("matching_results.xlsx", index=False)